In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tqdm import tqdm
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')

In [2]:
data=pd.read_csv("../input/sample/sample-2.csv")

In [3]:
data.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-007676813_00577,Complete,https://transcription.si.edu/transcribe/32203/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32203/image/NMAAHC-007676813_00577.jpg,1,"8 E encl\r\n\r\nI Anne H. Elliott, widow do so...",/32203/transcriptions/NMAAHC-007676813_00577.txt,\N,\N,32203,Oaths/Pardons,NaN,NaN
1,NMAAHC-007676818_00567,Complete,https://transcription.si.edu/transcribe/32796/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32796/image/NMAAHC-007676818_00567.jpg,1,I E C Whaley solemnly swear that I have carefu...,/32796/transcriptions/NMAAHC-007676818_00567.txt,\N,\N,32796,Oaths/Pardons,NaN,NaN
2,NMAAHC-007676815_00488-000001,Complete,https://transcription.si.edu/transcribe/32218/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32218/image/NMAAHC-007676815_00488-000001.jpg,1,43M\r\nI Mrs E Montgomery solemnly Swear that ...,/32218/transcriptions/NMAAHC-007676815_00488-0...,\N,\N,32218,Oaths/Pardons,NaN,NaN
3,NMAAHC-007677400_00375,Complete,https://transcription.si.edu/transcribe/26149/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/26149/image/NMAAHC-007677400_00375.jpg,1,"No. 2324\r\nI do solemnly swear or affirm, in ...",/26149/transcriptions/NMAAHC-007677400_00375.txt,\N,\N,26149,Oaths/Pardons,NaN,NaN
4,NMAAHC-007677378_00480,Complete,https://transcription.si.edu/transcribe/27345/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/27345/image/NMAAHC-007677378_00480.jpg,1,August 24. 1865\r\nAPPLICATION AND CERTIFICATE...,/27345/transcriptions/NMAAHC-007677378_00480.txt,\N,\N,27345,Oaths/Pardons,NaN,NaN


In [4]:
t0=data["transcription_text"][0]

In [5]:
t1=data["transcription_text"][1]

In [6]:
t2=data["transcription_text"][2]

# Preprocessing

In [7]:
text=data["transcription_text"]

In [8]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language="english")
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [9]:
stop_words.remove("of")

In [10]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
def process(s):
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [12]:
from tqdm import tqdm
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:02<00:00,  7.30it/s]


In [13]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 3872.86it/s]


In [14]:
text

0     8 E encl I Anne H Elliott widow solemnly swear...
1     I E C Whaley solemnly swear I carefully read A...
2     43M I Mrs E Montgomery solemnly Swear I carefu...
3     No 2324 I solemnly swear affirm presence of Al...
4     August 24 1865 APPLICATION AND CERTIFICATE OF ...
5     I Wm Whaley solemnly swear I carefully read Am...
6     I J F Steinmeyer solemnly swear I carefully re...
7     I Joseph Edings solemnly swear I carefully rea...
8     Vicksburg Miss July 22 1867 Thomas Andrew Oath...
9     455 RETAINED COPY I Jacob Schroder solemnly sw...
10    image eagle AMNESTY OATH Mississippi City Miss...
11    OATH I P P Bergevin solemnly swear I never vol...
12                                         Frank Conner
13    I Caroline Belitzer solemnly Swear I carefully...
14    Triplicate AMNESTY OATH Pass Christian Miss Au...
15    Retained Copy HEADQUARTERS UNITED STATES FORCE...
16                                      Robert A Wilson
17    APPLICATION AND CERTIFICATE OF James D Ken

In [15]:
t0

"8 E encl\r\n\r\nI Anne H. Elliott, widow do solemnly swear that I have read the Amnesty proclamation issued by Andrew Johnson President of the United States of America on May 29th 1865 and that I am not excepted from the benefits of that proclamation by any one of the fourteen exceptions therein made. Sworn to and subscribed before me at this 27th day of Novb 1865\r\nS. Wilde Durson\r\nMagt, DD\r\n\r\nA.H.E. [[image: 2 cent revenue stamp]] \r\n\r\nAnne H Elliott\r\n\r\nUnited States of America\r\nI Anne H Elliott, of the District of Beaufort, State of South Carolina do solemnly swear in the presence of Almighty God that I will henceforth faithfully support and defend the Constitution of the United States and the Union of the States thereunder, and that I will in like manner abide by and faithfully support all laws and proclamations which have been made during the existing Rebellion with reference to the Emancipation of Slaves- So help me God. \r\nSworn to & subscribed before me at thi

In [16]:
text[0]

'8 E encl I Anne H Elliott widow solemnly swear I read Amnesty proclamation issued Andrew Johnson President of United States of America May 29th 1865 I excepted benefit of proclamation one of fourteen exception therein made Sworn subscribed 27th day of Novb 1865 S Wilde Durson Magt DD AHE image 2 cent revenue stamp Anne H Elliott United States of America I Anne H Elliott of District of Beaufort State of South Carolina solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing Rebellion reference Emancipation of Slaves So help God Sworn subscribed 27th day of Novbr 1865 Magistrate S Wilde Durson AHE AHE AHE image 3 revenue stamp Anne H Elliott The named fair complexion grey hair Hazel eye 5 foot 3 inch high aged 63 year profession planter widow S Wilde Durson Magtr'

*extract transcription_text from the dataset*

In [17]:
text.head()

0    8 E encl I Anne H Elliott widow solemnly swear...
1    I E C Whaley solemnly swear I carefully read A...
2    43M I Mrs E Montgomery solemnly Swear I carefu...
3    No 2324 I solemnly swear affirm presence of Al...
4    August 24 1865 APPLICATION AND CERTIFICATE OF ...
Name: transcription_text, dtype: object

****

# Spacy

*SpaCy’s named entity recognition has been trained on the OntoNotes 5 corpus and it supports the following entity types:*

![](https://miro.medium.com/max/1400/1*qQggIPMugLcy-ndJ8X_aAA.png)

importing the necessary libraries

In [18]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

**trying to extract all useful information about the entity in text  from Doc(nlp(text) like : the entity | the lemmatization of the entity | the label of the entity and the explanation of the label**

In [19]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])

In [20]:
L2=[]
def NER1 (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L2.append([(word, word.ent_iob_, word.ent_type_, word.pos) for word in nlp_text])

In [21]:
NER(text)
NER1(text)

In [22]:
L1[0][:5]

[('Anne H Elliott', 11, 25, 'PERSON', 'People, including fictional'),
 ('Amnesty', 54, 61, 'ORG', 'Companies, agencies, institutions, etc.'),
 ('Andrew Johnson', 82, 96, 'PERSON', 'People, including fictional'),
 ('United States of America', 110, 134, 'GPE', 'Countries, cities, states'),
 ('May 29th 1865', 135, 148, 'DATE', 'Absolute or relative dates or periods')]

**structered the extracted information in a dataframe for using it in the next stage**

In [23]:
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])
        
df1=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()
   

,entities,start_char,end_char,label,explain,frequency
0,Anne H Elliott,11,25,PERSON,"People, including fictional",
1,Amnesty,54,61,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,82,96,PERSON,"People, including fictional",
3,United States of America,110,134,GPE,"Countries, cities, states",
4,May 29th 1865,135,148,DATE,Absolute or relative dates or periods,


In [24]:
df1.shape

(268, 6)

**adding the frequency of each entities in the text maybe it may help in improving the model later**

In [25]:
Counter(df1["entities"][:10])

Counter({'Anne H Elliott': 1,
         'Amnesty': 1,
         'Andrew Johnson': 1,
         'United States of America': 1,
         'May 29th 1865': 1,
         'one': 1,
         'fourteen': 1,
         'Sworn': 1,
         '27th day': 1,
         '1865': 1})

In [26]:
dict=Counter(df1["entities"])

In [27]:
for i in range(df1.shape[0]):
    df1["frequency"][i]=dict[df1["entities"].loc[i]]

************

****

In [28]:
df1[50:]

,entities,start_char,end_char,label,explain,frequency
50,43,0,2,CARDINAL,Numerals that do not fall under another type,1
51,Amnesty Proclamation,55,75,ORG,"Companies, agencies, institutions, etc.",7
52,ANDREW,83,89,ORG,"Companies, agencies, institutions, etc.",6
53,United States of America,111,135,GPE,"Countries, cities, states",12
54,May 29th 1865,136,149,DATE,Absolute or relative dates or periods,8
...,...,...,...,...,...,...
263,H F Holwell of,711,725,ORG,"Companies, agencies, institutions, etc.",1
264,31 day,747,753,DATE,Absolute or relative dates or periods,3
265,1865,761,765,DATE,Absolute or relative dates or periods,6
266,Frank E Miller,766,780,PERSON,"People, including fictional",2


In [29]:
Person=[]
ORG=[]
DATE=[]
GPE=[]

for i in range(df1.shape[0]):
    if df1.label.loc[i]=="PERSON":
        Person.append(df1.entities.loc[i]+": PERSON")
    elif df1.label.loc[i]=="ORG" :
        ORG.append(df1.entities.loc[i]+": ORG")
    elif df1.label.loc[i]=="DATE" :
        DATE.append(df1.entities.loc[i]+": DATE")
    elif df1.label.loc[i]=="GPE" :
        GPE.append(df1.entities.loc[i]+": GPE")

In [30]:
Person

['Anne H Elliott: PERSON',
 'Andrew Johnson: PERSON',
 'Anne H Elliott United States of America: PERSON',
 'Anne H Elliott: PERSON',
 'Hazel: PERSON',
 'Whaley: PERSON',
 'Andrew Johnson: PERSON',
 'Strikethrough: PERSON',
 'Oaths: PERSON',
 'Oaths: PERSON',
 'Hu Kennedy: PERSON',
 'Amnesty Oath Administered At: PERSON',
 'Hiserodt: PERSON',
 'William Whaley Sworn: PERSON',
 'William Whaley Sworn: PERSON',
 'Brown: PERSON',
 'Charleston Sc: PERSON',
 'County: PERSON',
 'JF Steinmeyer Sworn: PERSON',
 'Charleston Sc: PERSON',
 'Gry: PERSON',
 'Oaths: PERSON',
 'Joseph Edings: PERSON',
 'Thomas Andrew Oath of: PERSON',
 'Jacob Schroder: PERSON',
 'Jacob Schroder Sworn: PERSON',
 'Walhalla: PERSON',
 'Walhalla: PERSON',
 'Laborer: PERSON',
 'Amnesty Oath: PERSON',
 'Woodville Wilkinson: PERSON',
 'Frank Conner: PERSON',
 'Copy F K Field Capt: PERSON',
 'Triplicate AMNESTY: PERSON',
 'Hiram Smith: PERSON',
 'Hiram Smith: PERSON',
 'Pass Christian: PERSON',
 'Captn: PERSON',
 'Amnesty Oath:

In [31]:
ORG

['Amnesty: ORG',
 'Sworn: ORG',
 'AHE: ORG',
 'United States Union: ORG',
 'Sworn: ORG',
 'Magistrate S Wilde Durson AHE AHE AHE: ORG',
 'Amnesty Proclamation: ORG',
 'United State of America: ORG',
 'Whaley Sworn: ORG',
 'United States Union: ORG',
 'Whaley Sworn: ORG',
 'Department of State: ORG',
 'Amnesty Proclamation: ORG',
 'ANDREW: ORG',
 'Sgd Montgomery Sworn: ORG',
 'Levi Stuber: ORG',
 'United States Union: ORG',
 'Emancipation of Slaves: ORG',
 'Levi Stuber: ORG',
 'Department of State: ORG',
 'United States Union: ORG',
 'INTERREVENUE: ORG',
 'Sworn: ORG',
 'Amnesty Proclamation: ORG',
 'ANDREW: ORG',
 'United States Union of State: ORG',
 'Department of State: ORG',
 'Amnesty Proclamation: ORG',
 'ANDREW: ORG',
 'JF Steinmeyer Sworn: ORG',
 'United States Union: ORG',
 'Department of State: ORG',
 'Amnesty Proclamation: ORG',
 'ANDREW: ORG',
 'Joseph Edings Sworn: ORG',
 'United States Union: ORG',
 'Department of State: ORG',
 'Amnesty Proclamation: ORG',
 'ANDREW: ORG',


In [32]:
DATE

['May 29th 1865: DATE',
 '27th day: DATE',
 '1865: DATE',
 '27th day: DATE',
 '1865: DATE',
 '63 year: DATE',
 'May 29th 1865: DATE',
 '28 day: DATE',
 '28 day: DATE',
 'May 47 PM: DATE',
 '39 year: DATE',
 'May 29th 1865: DATE',
 '31 day: DATE',
 '1865: DATE',
 '31 day: DATE',
 '1865: DATE',
 '31st day: DATE',
 'August 24 1865: DATE',
 'May 29th 1865: DATE',
 '3 day of August 1865: DATE',
 'May 4: DATE',
 '3 day: DATE',
 '1865: DATE',
 'May 29th 1865: DATE',
 '16th day: DATE',
 '1474: DATE',
 '16th day: DATE',
 'May 29th 1865: DATE',
 '16 day of October 1865: DATE',
 '16 day of October 1865: DATE',
 'July 22: DATE',
 'July 22: DATE',
 '1867: DATE',
 'May 29th 1865: DATE',
 '6 day: DATE',
 '6 day: DATE',
 '29th day of September 1865: DATE',
 'Age 24 year: DATE',
 'Sept 29th 1865: DATE',
 '29th day: DATE',
 'May 29th 1865: DATE',
 '7th day: DATE',
 '7th day of September: DATE',
 '20th day of August 1865: DATE',
 '20th day of August 1865: DATE',
 'August 29th 1865: DATE',
 '22nd day of A

In [33]:
GPE

['United States of America: GPE',
 'South Carolina: GPE',
 'Charleston: GPE',
 'United States of America: GPE',
 'Charleston: GPE',
 'Charleston: GPE',
 'United States of America: GPE',
 'Charleston: GPE',
 'No1984 United States of America: GPE',
 'Charleston: GPE',
 'New Orleans: GPE',
 'US: GPE',
 'United States of America: GPE',
 'Charleston: GPE',
 'Charleston: GPE',
 'United States of America: GPE',
 'Charleston: GPE',
 'United States of America: GPE',
 'United States of America: GPE',
 'United States of America: GPE',
 'United States of America: GPE',
 'South Carolina: GPE',
 'Vicksburg: GPE',
 'United States of America: GPE',
 'South Carolina: GPE',
 'Mississippi City: GPE',
 'United States: GPE',
 'States: GPE',
 'Mississippi City: GPE',
 'Mississippi City: GPE',
 'United States: GPE',
 'United States: GPE',
 'United States: GPE',
 'United States: GPE',
 'United States: GPE',
 'Mississippi: GPE',
 'United States of America: GPE',
 'Charleston: GPE',
 'United States of America: 

In [34]:
type(GPE)

list

In [35]:
GPE=np.array(GPE)
ORG=np.array(ORG)
Person=np.array(Person)
DATE=np.array(DATE)

In [36]:
np.unique(GPE)

array(['Charleston: GPE', 'Gainsville: GPE', 'Mississippi City: GPE',
       'Mississippi: GPE', 'NC: GPE', 'New Orleans: GPE',
       'No1984 United States of America: GPE', 'South Carolina: GPE',
       'States: GPE', 'US: GPE', 'United States of America: GPE',
       'United States: GPE', 'Vicksburg: GPE'], dtype='<U36')

In [37]:
np.unique(ORG)

array(['AHE: ORG', 'AMNESTY: ORG', 'ANDREW: ORG',
       'Amnesty Proclamation: ORG', 'Amnesty: ORG', 'Belitzer Sworn: ORG',
       'Bray: ORG', 'Charleston State of So Carolina: ORG',
       'Color of Eyes Blue Color of Hair Dark Characteristics: ORG',
       'Department of State: ORG', 'Emancipation of Slaves: ORG',
       'Eyes Blue Color of Hair: ORG', 'H F Holwell of: ORG',
       'H Mathews Capt: ORG', 'INTERREVENUE: ORG',
       'JF Steinmeyer Sworn: ORG', 'Joseph Edings Sworn: ORG',
       'Levi Stuber: ORG', 'Magistrate S Wilde Durson AHE AHE AHE: ORG',
       'OATH I P P Bergevin: ORG',
       'P H Joor Justice of Peace Wilkinson County State of: ORG',
       'Rebellion reference Emancipation of Slaves: ORG',
       'Regt U S C Infy Provost Marshal: ORG',
       'Sgd Montgomery Sworn: ORG', 'Sworn: ORG',
       'UNITED STATES FORCES No 1282 PROVOST MARSHALS OFFICE: ORG',
       'US Col Infty Provost Marshal: ORG',
       'USCI Commanding DUPLICATE: ORG',
       'USCI Commandi

In [38]:
len(np.unique(Person))

38

In [39]:
np.unique(Person)

array(['Amnesty Oath Administered At: PERSON', 'Amnesty Oath: PERSON',
       'Andrew Johnson: PERSON',
       'Anne H Elliott United States of America: PERSON',
       'Anne H Elliott: PERSON', 'Bolt Capt 54th: PERSON',
       'Brown: PERSON', 'Captn Commanding TRIPLICATE: PERSON',
       'Captn: PERSON', 'Charleston Sc: PERSON', 'City Miss: PERSON',
       'Copy F K Field Capt: PERSON', 'County: PERSON',
       'Frank Conner: PERSON', 'Frank E Miller: PERSON',
       'Gibbs Henderson: PERSON', 'Gry: PERSON', 'Hazel: PERSON',
       'Hiram Smith: PERSON', 'Hiserodt: PERSON', 'Hu Kennedy: PERSON',
       'JF Steinmeyer Sworn: PERSON', 'Jacob Schroder Sworn: PERSON',
       'Jacob Schroder: PERSON',
       'James D Kendall Amnesty Oath Administered At Oacen Springs: PERSON',
       'Joseph Edings: PERSON', 'Laborer: PERSON', 'Oaths: PERSON',
       'Pass Christian: PERSON', 'Retained Copy: PERSON',
       'Robert A: PERSON', 'Strikethrough: PERSON',
       'Thomas Andrew Oath of: PERSON

In [40]:
np.unique(DATE)

array(['1474: DATE', '16 day of October 1865: DATE', '16th day: DATE',
       '1865 1865: DATE', '1865: DATE', '1867: DATE',
       '20th day of August 1865: DATE', '22nd day of August 1865: DATE',
       '27th day: DATE', '28 day: DATE',
       '29th day of September 1865: DATE', '29th day: DATE',
       '3 day of August 1865: DATE', '3 day: DATE', '31 day: DATE',
       '31st day: DATE', '39 year: DATE', '6 day: DATE', '63 year: DATE',
       '7th day of September: DATE', '7th day: DATE', 'Age 24 year: DATE',
       'August 24 1865: DATE', 'August 29th 1865: DATE', 'July 22: DATE',
       'May 29th 1865: DATE', 'May 47 PM: DATE', 'May 4: DATE',
       'Miss 31st day: DATE', 'Sept 29th 1865: DATE'], dtype='<U32')

****

In [41]:
t0

"8 E encl\r\n\r\nI Anne H. Elliott, widow do solemnly swear that I have read the Amnesty proclamation issued by Andrew Johnson President of the United States of America on May 29th 1865 and that I am not excepted from the benefits of that proclamation by any one of the fourteen exceptions therein made. Sworn to and subscribed before me at this 27th day of Novb 1865\r\nS. Wilde Durson\r\nMagt, DD\r\n\r\nA.H.E. [[image: 2 cent revenue stamp]] \r\n\r\nAnne H Elliott\r\n\r\nUnited States of America\r\nI Anne H Elliott, of the District of Beaufort, State of South Carolina do solemnly swear in the presence of Almighty God that I will henceforth faithfully support and defend the Constitution of the United States and the Union of the States thereunder, and that I will in like manner abide by and faithfully support all laws and proclamations which have been made during the existing Rebellion with reference to the Emancipation of Slaves- So help me God. \r\nSworn to & subscribed before me at thi

In [42]:
text[0]

'8 E encl I Anne H Elliott widow solemnly swear I read Amnesty proclamation issued Andrew Johnson President of United States of America May 29th 1865 I excepted benefit of proclamation one of fourteen exception therein made Sworn subscribed 27th day of Novb 1865 S Wilde Durson Magt DD AHE image 2 cent revenue stamp Anne H Elliott United States of America I Anne H Elliott of District of Beaufort State of South Carolina solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing Rebellion reference Emancipation of Slaves So help God Sworn subscribed 27th day of Novbr 1865 Magistrate S Wilde Durson AHE AHE AHE image 3 revenue stamp Anne H Elliott The named fair complexion grey hair Hazel eye 5 foot 3 inch high aged 63 year profession planter widow S Wilde Durson Magtr'

In [43]:
L1[:1]

[[('Anne H Elliott', 11, 25, 'PERSON', 'People, including fictional'),
  ('Amnesty', 54, 61, 'ORG', 'Companies, agencies, institutions, etc.'),
  ('Andrew Johnson', 82, 96, 'PERSON', 'People, including fictional'),
  ('United States of America', 110, 134, 'GPE', 'Countries, cities, states'),
  ('May 29th 1865', 135, 148, 'DATE', 'Absolute or relative dates or periods'),
  ('one',
   184,
   187,
   'CARDINAL',
   'Numerals that do not fall under another type'),
  ('fourteen',
   191,
   199,
   'CARDINAL',
   'Numerals that do not fall under another type'),
  ('Sworn', 223, 228, 'ORG', 'Companies, agencies, institutions, etc.'),
  ('27th day', 240, 248, 'DATE', 'Absolute or relative dates or periods'),
  ('1865', 257, 261, 'DATE', 'Absolute or relative dates or periods'),
  ('AHE', 285, 288, 'ORG', 'Companies, agencies, institutions, etc.'),
  ('2 cent', 295, 301, 'MONEY', 'Monetary values, including unit'),
  ('Anne H Elliott United States of America',
   316,
   355,
   'PERSON',
   

In [44]:
sentence = [x for x in nlp(text[0]).sents]
displacy.render(nlp(str(sentence)) , style='ent')

In [45]:
t1

'I E C Whaley solemnly swear that I have carefully read the Amnesty\r\nProclamation issued by Andrew Johnson, President of the United State of America, on May 29th, 1865, and that I am not excepted from the benefits of the Proclamation by any one of the fourteen exceptions therein made [[Strikethrough]] except [[strikethrough/]] \r\nE C Whaley\r\nSworn to and subscribed before me at Charleston SC this 28\r\nday of Sept 1865\r\n\r\nLevi Steuben\r\nMaj. 47th P.M.\r\n\r\nUnited States of America\r\nE C Whaley of the city [[strikethrough]] county [[strikethrough/]] Charleston Sate of South Ca do solemnly swear in the presence of the Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves-SO HELP ME GOD\r\n\r\nE C Wha

In [46]:
sentence1 = [x for x in nlp(text[1]).sents]
displacy.render(nlp(str(sentence1)) , style='ent')

In [47]:
t2

'43M\r\nI Mrs E Montgomery solemnly Swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\nSgd in Montgomery\r\n\r\nSworn to and subscribed before me at Charleston SC this 31 day of Oct 1865.\r\nLevi Stuber\r\nMajor 47th Pa. Vet. Vols. and Provost Marshal.\r\n\r\n[[left margin]] Duplicate [[/left margin]]\r\n\r\nNo.1984\r\nUnited States of America.\r\nI Mrs E. Montgomery of the [[strikethrough]] County [[/strikethrough]]mnly Swear in the presence of Almighty God, that I will henceforth faithfully support and defend the constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves

In [48]:
sentences = [x for x in nlp(text[2]).sents]
displacy.render(nlp(str(sentences)) , style='ent')

In [49]:
np.unique(DATE)

array(['1474: DATE', '16 day of October 1865: DATE', '16th day: DATE',
       '1865 1865: DATE', '1865: DATE', '1867: DATE',
       '20th day of August 1865: DATE', '22nd day of August 1865: DATE',
       '27th day: DATE', '28 day: DATE',
       '29th day of September 1865: DATE', '29th day: DATE',
       '3 day of August 1865: DATE', '3 day: DATE', '31 day: DATE',
       '31st day: DATE', '39 year: DATE', '6 day: DATE', '63 year: DATE',
       '7th day of September: DATE', '7th day: DATE', 'Age 24 year: DATE',
       'August 24 1865: DATE', 'August 29th 1865: DATE', 'July 22: DATE',
       'May 29th 1865: DATE', 'May 47 PM: DATE', 'May 4: DATE',
       'Miss 31st day: DATE', 'Sept 29th 1865: DATE'], dtype='<U32')

**extracting sentences with spacy and their interest in NER**

In [50]:
doc=nlp(text[1])
sentences=list(doc.sents)

In [51]:
len(sentences)

3

In [52]:
for sent in sentences:
    print(type(sent))
    print(sent)

<class 'spacy.tokens.span.Span'>
I E C Whaley solemnly swear I carefully read Amnesty Proclamation issued Andrew Johnson President of United State of America May 29th 1865
<class 'spacy.tokens.span.Span'>
I excepted benefit of Proclamation one of fourteen exception therein made Strikethrough except strikethrough E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben Maj 47th PM United States of America E C Whaley of city strikethrough county strikethrough Charleston Sate of South Ca solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of SlavesSO HELP ME GOD E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben May 47 PM
<class 'spacy.tokens.span.Span'>
The fair complexion light hair gray eye 5 foot 10 inch high 39 year profession planter The original Oaths trans

In [53]:
for word in doc.ents:
    print(word.text, word.start_char, word.end_char, word.label_)
    

Whaley 6 12 PERSON
Amnesty Proclamation 45 65 ORG
Andrew Johnson 73 87 PERSON
United State of America 101 124 ORG
May 29th 1865 125 138 DATE
one 174 177 CARDINAL
fourteen 181 189 CARDINAL
Strikethrough 213 226 PERSON
Whaley Sworn 252 264 ORG
Charleston 276 286 GPE
28 day 290 296 DATE
47th 327 331 ORDINAL
United States of America 335 359 GPE
Charleston 414 424 GPE
United States Union 537 556 ORG
Emancipation 668 680 PRODUCT
Whaley Sworn 709 721 ORG
Charleston 733 743 GPE
28 day 747 753 DATE
May 47 PM 780 789 DATE
eye 5 foot 10 inch 826 844 QUANTITY
39 year 850 857 DATE
Oaths 890 895 PERSON
Department of State 930 949 ORG
Left Margin Copy Levi Steuben 1040 1069 WORK_OF_ART
47th 1074 1078 ORDINAL


**Similarity**

In [54]:
nlp = spacy.load("en_core_web_lg")
tokens = nlp(text[1])

*Spacy also provides inbuilt integration of dense, real valued vectors representing distributional similarity information.
Models that come with built-in word vectors make them available as the Token.vector attribute. Doc.vector and Span.vector will default to an average of their token vectors. You can also check if a token has a vector assigned, and get the L2 norm, which can be used to normalize vectors.*

awesome!

****

****

In [55]:
from keras.preprocessing import text, sequence
tokenizer = text.Tokenizer()

In [56]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=7dca9e7b14976963bca2910935a905d7d089e114fa134528aadce5c8e20c3d62
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.19.0-py3-none-any.whl size=19553 sha256=e9c5992269cd1078e2d3b01f20774ae39fccb9bfb20748f2b5018a3ffdfa801f
  Stored in directory: /root/.cache/pip/wheels/fe/5d/69/ff53a908b9f14fb7730a58fdede0fac4cdc

In [57]:
!conda uninstall tokenizers, transformers
!pip install transformers

Solving environment: - \ | / - \ | / - \ | / - \ | failed

PackagesNotFoundError: The following packages are missing from the target environment:
  - tokenizers,
  - transformers




In [58]:

!pip install tokenizers
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 4.19.2 which is incompatible.


****

****

********

# Hugging_Face Pipelines

**it was a mess up** Xd

In [59]:
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cpu requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.11.0+cpu requires torch==1.11.0, but you have torch 1.4.0 which is incompatible.
pytorch-lightning 1.6.3 requires torch>=1.8.*, but you have torch 1.4.0 which is incompatible.
kornia 0.5.8 requires torch>=1.6.0, but you have torch 1.4.0 which is incompatible.
fastai 2.6.3 requires torch<1.12,>=1.7.0, but you have torch 1.4.0 which is incompatible.
fairscale 0.4.6 requires torch>=1.8.0, but you have torch 1.4.0 which is incompatible.
allennlp 2.9.3 requires torch<1.12.0,>=1.6.0, but you have torch 1.4.0 which is incompatible.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 4.19.2 which is incomp

In [60]:
from transformers import pipeline
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from transformers import AutoModel, AutoTokenizer

In [61]:
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english" 
model = AutoModel.from_pretrained(model_name)
tokenizerr = AutoTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

**AutoTokenizer**

*This is a generic tokenizer class that will be instantiated as one of the tokenizer classes of the library when created with the AutoTokenizer.from_pretrained() class method*

In [62]:
tokens = tokenizerr(data["transcription_text"][0], return_tensors="pt")

In [63]:
ner_pipe = pipeline(task='ner', model=model_name,tokenizer=tokenizerr)

2022-06-02 10:00:20.331838: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [64]:
L=[]
for ent in ner_pipe(data["transcription_text"][0]):
    L.append(ent)
L[1]

{'entity': 'I-PER',
 'score': 0.9883646,
 'index': 8,
 'word': 'H',
 'start': 16,
 'end': 17}

*thanks god (after 2hours of unsigned  error)*

In [65]:
Per=[]
ORg=[]
GPe=[]

for i in range(50):
    if ( L[i]["entity"]=='I-PER') | (L[i]["entity"]=="B-PER") | (L[i]["entity"]=="O-PER"):
        Per.append(L[i]["word"]+": PERSON")
    elif (L[i]["entity"]=="I-LOC")| (L[i]["entity"]=="B-LOC" )|( L[i]["entity"]=="O-LOC"):
        GPe.append(L[i]["word"]+": Location")
    elif (L[i]["entity"]=="I-MISC")| (L[i]["entity"]=="B-MISC") | (L[i]["entity"]=="O-MISC"):
        ORg.append(L[i]["word"]+": organization")
   

****

**trying other model ! it's getting better**

In [66]:
pipe = pipeline(task='ner', model="dslim/bert-base-NER-uncased")

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [67]:
L2=[]
for ent in pipe(data["transcription_text"][0]):
    L2.append(ent)
len(L2)

42

In [68]:
Per=[]
ORg=[]
GPe=[]

for i in range(42):
    if ( L2[i]["entity"]=='I-PER') | (L2[i]["entity"]=="B-PER") | (L2[i]["entity"]=="O-PER"):
        Per.append(L2[i]["word"]+": PERSON")
    elif (L2[i]["entity"]=="I-LOC")| (L2[i]["entity"]=="B-LOC" )|( L2[i]["entity"]=="O-LOC"):
        GPe.append(L2[i]["word"]+": Location")
    elif (L2[i]["entity"]=="I-MISC")| (L2[i]["entity"]=="B-MISC") | (L2[i]["entity"]=="O-MISC"):
        ORg.append(L2[i]["word"]+": organization")

****

****

# Spacy with lg model

In [69]:
import spacy
from spacy import displacy
import en_core_web_lg
import en_core_web_sm
nlp1 = en_core_web_lg.load()

In [70]:
L1=[]
def NER3 (txt):
    for i in range(20):
        textt =  nlp1(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in textt.ents])

In [71]:
NER3(data["transcription_text"])

In [72]:
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])
        
df3=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df3.head()

,entities,start_char,end_char,label,explain,frequency
0,8,0,1,CARDINAL,Numerals that do not fall under another type,
1,Anne H Elliott,11,25,PERSON,"People, including fictional",
2,Amnesty,54,61,PERSON,"People, including fictional",
3,Andrew Johnson,82,96,PERSON,"People, including fictional",
4,United States of America,110,134,GPE,"Countries, cities, states",


In [73]:
Person1=[]
ORG1=[]
DATE1=[]
GPE1=[]

for i in range(df1.shape[0]):
    if df3.label.loc[i]=="PERSON":
        Person1.append(df3.entities.loc[i]+": PERSON")
    elif df3.label.loc[i]=="ORG" :
        ORG1.append(df3.entities.loc[i]+": ORG")
    elif df3.label.loc[i]=="DATE" :
        DATE1.append(df3.entities.loc[i]+": DATE")
    elif df3.label.loc[i]=="GPE" :
        GPE1.append(df3.entities.loc[i]+": GPE")

In [74]:
GPE1=np.array(GPE1)
ORG1=np.array(ORG1)
Person1=np.array(Person1)
DATE1=np.array(DATE1)


In [75]:
nb=0
for i in range(47):
    if GPE[i] ==GPE1[i]:
        nb=nb+1
print(nb)

9


****

# Flair

In [76]:
!pip install flair

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [77]:
T=[]
from flair.data import Sentence
from flair.models import SequenceTagger
from segtok.segmenter import split_single
tagger=SequenceTagger.load('ner-ontonotes')

for i in range(20):
    text=data["transcription_text"][i]
    sentence=[Sentence(sent,use_tokenizer=True)for sent in split_single(text)]
    tagger.predict(sentence)
    for sent in sentence:
            for entity in sent.get_spans('ner'):
                 T.append(entity)

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

2022-06-02 10:03:03,908 loading file /root/.flair/models/ner-english-ontonotes/f46dcd14689a594a7dd2a8c9c001a34fd55b02fded2528410913c7e88dbe43d4.1207747bf5ae24291205b6f3e7417c8bedd5c32cacfb5a439f3eff38afda66f7
2022-06-02 10:03:11,412 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [78]:
T

[Span[4:7]: "Anne H Elliott" → PERSON (0.8023),
 Span[12:13]: "Amnesty" → ORG (0.3231),
 Span[15:17]: "Andrew Johnson" → PERSON (0.9016),
 Span[19:23]: "United States of America" → ORG (0.6651),
 Span[23:26]: "May 29th 1865" → DATE (0.9434),
 Span[33:34]: "fourteen" → CARDINAL (0.9944),
 Span[39:41]: "27th day" → DATE (0.7263),
 Span[42:50]: "Novb 1865 S Wilde Durson Magt DD AHE" → WORK_OF_ART (0.5594),
 Span[51:53]: "2 cent" → MONEY (0.8272),
 Span[55:58]: "Anne H Elliott" → PERSON (0.7771),
 Span[58:62]: "United States of America" → GPE (0.5298),
 Span[63:66]: "Anne H Elliott" → PERSON (0.8255),
 Span[67:71]: "District of Beaufort State" → ORG (0.6853),
 Span[72:74]: "South Carolina" → GPE (0.9963),
 Span[85:92]: "Constitution of United States Union of States" → LAW (0.8941),
 Span[113:115]: "27th day" → DATE (0.7794),
 Span[120:122]: "Wilde Durson" → PERSON (0.7184),
 Span[126:127]: "3" → CARDINAL (0.6127),
 Span[129:132]: "Anne H Elliott" → PERSON (0.803),
 Span[140:144]: "5 foot 3